In [8]:
import os
import yaml
import random
import numpy as np
import pickle as pkl
import numpy as np
import pandas as pd

In [5]:
class Config:
    def __init__(self, filename):
        with open(filename, 'r') as file:
            configdict = yaml.safe_load(file)
        self.__dict__.update(configdict)
config = Config("../configs/config-intact0.yaml")

In [6]:
config.__dict__

{'networkfolder': 'data/intact',
 'gofolder': 'data/go',
 'species': ['fly', 'mouse', 'rat', 'bakers'],
 'beta': 0.01,
 'gamma': 0.25,
 'r_iter': 10,
 'isorank_alpha': 0.6,
 'output_cluster': 'data/clusters/R0-intact-cluster-0.6.pkl'}

In [12]:
gomaps = {}
combinedgo = pd.DataFrame()
for sp in config.species:
    gofile = f"../{config.gofolder}/{sp}.output.mapping.gaf"
    gomaps[sp] = pd.read_csv(gofile, sep = "\t").loc[:, ["GO", "type", "swissprot"]]
    combinedgo = pd.concat([combinedgo, gomaps[sp]])

## GO files; which is a TSV file, one GO term can have more than one proteins

In [76]:
cgos = {}
cgos["molecular_function"] = combinedgo.loc[combinedgo["type"] == "molecular_function", ["GO", "swissprot"]]
cgos["biological_process"] = combinedgo.loc[combinedgo["type"] == "biological_process", ["GO", "swissprot"]]
cgos["cellular_component"] = combinedgo.loc[combinedgo["type"] == "cellular_component", ["GO", "swissprot"]]
cgos["molecular_function"]

,GO,swissprot
0,GO:0000009,Q9VH78
1,GO:0000009,Q9VH78
2,GO:0000009,Q9V7W1
3,GO:0000010,Q9VP87
4,GO:0000010,Q8SY08
...,...,...
13966,GO:1990883,P53914
13967,GO:1990932,Q07953
13968,GO:1990948,P40577
13969,GO:1990948,Q8TGU5


In [77]:
cgosupdated = {}
for ind, df in cgos.items():
    cgosupdated[ind] = df.groupby("swissprot").aggregate(list)

## Entropy computation, implementation from Isorank-n

In [86]:
from collections import defaultdict

def getentropy(cluster, gomap, norm_clustersize = False):
    def entropy(probs):
        probs = np.array(probs)
        logprobs = np.log(np.where(probs == 0, 1, probs))
        return -np.sum(probs * logprobs)
    allgos = []
    goprots = defaultdict(set)
    probs = []
    nprots = len(cluster)
    for prot in cluster:
        if prot in gomap.index:
            gos = gomap.loc[prot, "GO"]
            for go in gos:
                goprots[go].add(prot)
    for go, prots in goprots.items():
        probs.append(len(prots) / nprots)
    if len(probs) == 0:
        return np.nan
    entr = entropy(probs)
    if norm_clustersize:
        entr = entr / len(goprots)
    return entr

def getavgentropy(clusters, gomap, norm_clustersize = False):
    entropy = []
    for ref, items in clusters.items():
        clust = set(items)
        clust.add(ref)
        entropy.append(getentropy(clust, gomap, norm_clustersize))
    entropy = np.array(entropy)
    entropy = entropy[~np.isnan(entropy) ]
    return np.mean(entropy)

## Get the clusters: The clusters are pickled dictionary, where the key is the representative protein and the values are the other proteins clustered to it

In [93]:
R0file = f"../{config.output_cluster}"
with open(R0file, "rb") as rf:
    R0clust = pkl.load(rf)
R1file = f"../data/clusters/R1-intact-cluster-0.6.pkl"
with open(R1file, "rb") as rf:
    R1clust = pkl.load(rf)

## Compute Entropy; Both Normalized and Un-Normalized (from Isorank-n paper)

In [96]:
entropyresults = []
for R, Rname in [(R0clust, "R0"), (R1clust, "R1")]:
    for go in ["molecular_function", "biological_process", "cellular_component"]:
        for normalize in [True, False]:
            entr = getavgentropy(R, cgosupdated[go], norm_clustersize = normalize)
            entropyresults.append((Rname, go, "normalized" if normalize else "unnormalized", entr))

In [109]:
dfentropy = pd.DataFrame(entropyresults, columns = ["IsoRank", "GO", "IsNormalized", "Score"])
dfentropy = dfentropy.iloc[:, [2, 1, 0, 3]]
dfentropy

,IsNormalized,GO,IsoRank,Score
0,normalized,molecular_function,R0,0.984787
1,unnormalized,molecular_function,R0,0.854999
2,normalized,biological_process,R0,1.479757
3,unnormalized,biological_process,R0,2.714411
4,normalized,cellular_component,R0,1.019729
5,unnormalized,cellular_component,R0,0.922933
6,normalized,molecular_function,R1,1.009916
7,unnormalized,molecular_function,R1,0.939822
8,normalized,biological_process,R1,1.540529
9,unnormalized,biological_process,R1,3.037635


In [112]:
dfentropy = dfentropy.sort_values(["IsNormalized", "GO"])
dfecomp  = pd.pivot(dfentropy, index=["IsNormalized", "GO"], columns="IsoRank", values = "Score")
dfecomp                    

IsoRank                                 R        R0        R1
IsNormalized GO                                              
normalized   biological_process  1.546539  1.479757  1.540529
             cellular_component  1.029363  1.019729  1.029413
             molecular_function  1.004409  0.984787  1.009916
unnormalized biological_process  3.068622  2.714411  3.037635
             cellular_component  1.020900  0.922933  1.006943
             molecular_function  0.954223  0.854999  0.939822

In [113]:
dfecomp.to_csv("isorankn_entropy.tsv", sep = "\t")